In [ ]:
import csv
import json
import pandas as pd
import re
import logging
import os
import random
import sys
from heapq import nlargest
import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd


with open('../ScientificWritingDataset/authorJsonAndPaperJson/author_json.json', 'r',encoding='utf-8') as f:
    authors_sampled = json.load(f)
print(len(authors_sampled))

with open('../ScientificWritingDataset/authorJsonAndPaperJson/paper_json.json', 'r',encoding='utf-8') as f:
    papers_sampled = json.load(f)
print(len(papers_sampled))

In [ ]:
# sampled papers to 500 
count = 0
max_paper = 100
author_paper_count = []
paper_author_count = []
papers_sampled = {}
authors_sampled = {}
for i in papers:
    if count < 100:
        flag = True
        for author in papers[i]["authors"]:
            if len(authors[author]["papers"])<5:
                flag = False
        if flag and len(papers[i]["authors"]) > 1:
            paper_author_count.append(len(papers[i]["authors"]))
            papers_sampled[i] = papers[i]
            papers_sampled[i]["author_history"] = {}
            
            for author in papers[i]["authors"]:
                papers_sampled[i]["author_history"][author] = {}
                papers_sampled[i]["author_history"][author]["authors_history_text"] = [papers[i]["abstract"]]
                papers_sampled[i]["author_history"][author]["authors_history_dict"] = []
                paper_count_tmp = 0
                for paper_tmp in authors[author]["papers"]:
                    if paper_count_tmp < max_paper and paper_tmp["title"] != papers[i]["title"]:
                        papers_sampled[i]["author_history"][author]["authors_history_text"].append("Abstract: "+paper_tmp["abstract"]+"\nTitle: "+paper_tmp["title"])
                        papers_sampled[i]["author_history"][author]["authors_history_dict"].append({"Abstract":paper_tmp["abstract"],"Title":paper_tmp["title"]})
                        paper_count_tmp = paper_count_tmp + 1
                author_paper_count.append(len(authors[author]["papers"]))
                authors_sampled[author] = authors[author]
            count = count + 1
print("papers count after sampled: ",len(papers_sampled))
print("authors count after sampled: ",len(authors_sampled))


In [ ]:
print("author to papers: ", min(author_paper_count),max(author_paper_count),sum(author_paper_count)/len(author_paper_count))
print("paper to authors: ", min(paper_author_count),max(paper_author_count),sum(paper_author_count)/len(paper_author_count))

for i in papers_sampled:
    print(papers_sampled[i]["title"])
    print(papers_sampled[i]["author_history"]["1705375"]["authors_history_text"])
    break

In [ ]:
import concurrent.futures
import csv
import json
import logging
import os
import random
import sys
from heapq import nlargest
import random
import openai
import time
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
from transformers import AutoModel, AutoTokenizer

TOP_K = 20
task_num = "1"
i = 0
error_list=[]
openai.api_key = ""
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/")
]


# Contriever part
tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

# Mean pooling
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings


def construct_prompt_for_user_profile(evidences,profile_type):
    prompt_str = ""
    if profile_type == "topics":
        evidence_str = ""
        prompt_str = "Given the following scientific paper writing history, Use a few words to describe the topics that could represent user's interests:\n"
        for index, evidence in enumerate(evidences):
            evidence_str = "History #"+str(index) +"\nAbstract: "+evidence["Abstract"]+"\nTitle: "+evidence["Title"]
        prompt_str = prompt_str + evidence_str +"\nTopics:"
    if profile_type == "keywords":
        evidence_str = ""
        prompt_str = "Given the following scientific paper writing history, extract the top common keywords in the titles that may represent users' interests:\n"	
        for index, evidence in enumerate(evidences):	
            evidence_str = "History #"+str(index) +"\nAbstract: "+evidence["Abstract"]+"\nTitle: "+evidence["Title"]
        prompt_str = prompt_str + evidence_str +"\nKeywords:"
    return prompt_str

def run_openai(prompt,key_bundles,i):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        print(indice+"\t"+result+"\n")
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            print(indice+"\t"+result+"\n")
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                print(indice+"\t"+result+"\n")
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
        #print(result)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
        #print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result
authorDict = {}

with open("naive_method_v3.0/author_profiling_"+str(TOP_K)+"_examples_.tsv", "wb", buffering=0) as out_file:
    for indice in papers_sampled:
        for authorId in papers_sampled[indice]["author_history"]:
            if authorId not in authorDict:
                top_score_docs = random.sample(papers_sampled[indice]["author_history"][authorId]["authors_history_dict"], min(TOP_K,len(papers_sampled[indice]["author_history"][authorId]["authors_history_dict"])))
                # build prompt with history and question 
                prompt = construct_prompt_for_user_profile(top_score_docs,"keywords")
                keywords = run_openai(prompt,key_bundles,i)
                # time.sleep(2)
                i = i +1
                prompt = construct_prompt_for_user_profile(top_score_docs,"topics")
                topics = run_openai(prompt,key_bundles,i)
                i = i +1
                write_str = bytes(str(authorId)+"\t"+keywords+"\t"+topics+"\n", 'utf-8')
                out_file.write(write_str)
                authorDict[authorId] = 1